In [1]:
import sys
sys.path.append(r'C:\Users\xzhu\Documents\GitHub\trading')

import pandas as pd
import numpy as np
import datetime
import math
from pytorch_forecasting import TimeSeriesDataSet
from zenzic.data.watchlist import SP500

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_forecasting.data.encoders import TorchNormalizer
from madgrad import MADGRAD

In [2]:
%load_ext autoreload
%autoreload 2

from zenzic.strategies.pytorch.data.trades import load_wl_trades, load_quotes, Dataset
from zenzic.strategies.pytorch.data.utils import load_hist_quotes
from zenzic.strategies.pytorch.autoformer.trade.models import AEoT, AoT

In [3]:
train = Dataset(r'C:\Trading\CCI-RSI\2022-02-11\samples.pkl', 'train', 256)

In [30]:
class Configs(dict):
    pass

configs = Configs()
configs.seq_len = 256
configs.output_attention = False
configs.enc_in = 4
configs.d_model = 1024
configs.embed = 'timeF'
configs.freq = 'd'
configs.dropout = 0.3
configs.e_layers = 2
configs.factor = 4
configs.n_heads = 3
configs.d_ff = 389
configs.moving_avg = 21 # must be odd number.
configs.activation = 'gelu'
configs.learning_rate = 1e-3

m = AEoT(configs)
x = []
x_time_enc = []
y = []
for i in range(0, 16):
    a, b, c, _ = train[i]
    x.append(np.float32(a))
    x_time_enc.append(np.float32(b))
    y.append(np.float32(c))
x = np.array(x)
x_time_enc = np.array(x_time_enc)
y = np.array(y)
m(torch.from_numpy(x), torch.from_numpy(x_time_enc)).shape



torch.Size([16])

In [31]:
class Configs(dict):
    pass

configs = Configs()
configs.seq_len = 256
configs.output_attention = False
configs.enc_in = 4
configs.d_model = 1024
configs.embed = 'timeF'
configs.freq = 'd'
configs.dropout = 0.3
configs.e_layers = 2
configs.factor = 4
configs.n_heads = 3
configs.d_ff = 389
configs.moving_avg = 21 # must be odd number.
configs.activation = 'gelu'
configs.label_len = 64
configs.pred_len = 1
configs.dec_in = 4
configs.d_layers = 1
configs.c_out = 4
configs.learning_rate = 1e-3

m = AoT(configs)
x = []
x_time_enc = []
y = []
y_time_enc = []
for i in range(0, 16):
    a, b, c, d = train[i]
    x.append(np.float32(a))
    x_time_enc.append(np.float32(b))
    y.append(np.float32(c))
    y_time_enc.append(np.float32(d))
x = np.array(x)
x_time_enc = np.array(x_time_enc)
y = np.array(y)
y_time_enc = np.array(y_time_enc)
m(torch.from_numpy(x), torch.from_numpy(x_time_enc), torch.from_numpy(y_time_enc)).shape

torch.Size([16])